# **DATOS CIENTÍFICOS**

**ITBA - Maestría en Ciencia de Datos - 2023**

**Trabajo Práctico - Alen Jiménez**

- El objetivo de esta notebook es clasificar datos provenientes de sensores de ondas cerebrales, que se estructuran en series de tiempo.
- Las ondas cerebrales que van a ser estudiadas se corresponden con dos comportamientos: "ojos abiertos" y "pestañeos".
- Se va a intentar construir un modelo de clasificación que pueda distinguir estos dos comportamientos a partir de los datos provistos por las ondas cerebrales.
- El análisis descriptivo de todas las series disponibles se encuentra en la notebook analisis_filtros.ipynb

# Tabla de Contenidos
* [Set Up](#setup)

# Set Up <a class = 'anchor' id = 'setup'></a>

In [3]:
# Importamos bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from io import StringIO
import os
import sys, select
import time
import datetime
import math
from scipy import stats
from scipy.fftpack import fft
from scipy.signal import firwin, remez, kaiser_atten, kaiser_beta
from scipy.signal import butter, filtfilt, buttord
from scipy.signal import butter, lfilter
from scipy.signal import find_peaks
from collections import Counter
#from xgboost import XGBClassifier # Clasificador de XGBoost
#from bayes_opt import BayesianOptimization # Optimización Bayesiana
from sklearn.model_selection import cross_val_score, train_test_split # Cross Validation y partición del dataset en Train y Test
from sklearn.metrics import accuracy_score # Para medición del accuracy una vez que hagamos el XGBoost con los mejores parametros que nos da la optimización


In [3]:
# Directorio de trabajo

directorio_de_trabajo = 'C:/itba_datos_geograficos/ramele/tp'
os.chdir(directorio_de_trabajo)
print(f'Directorio actual de trabajo: {os.getcwd()}')

Directorio actual de trabajo: C:\itba_datos_geograficos\ramele\tp


In [9]:
# Importamos los archivos

archivo1 = 'ojosabiertos'
archivo2 = 'pestaneos'

df1 = pd.read_csv('datos/%s.dat' % archivo1
                  , delimiter = ' '
                  , names = ['timestamp','counter','eeg','attention','meditation','blinking'])

df2 = pd.read_csv('datos/%s.dat' % archivo2
                  , delimiter = ' '
                  , names = ['timestamp','counter','eeg','attention','meditation','blinking'])

In [22]:
df1.shape

(30850, 6)

In [23]:
df2.shape

(30840, 6)

In [10]:
# Medida: Densidad Espectral

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def psd(y):
    # Number of samplepoints
    N = 512
    # sample spacing
    T = 1.0 / 512.0
    # From 0 to N, N*T, 2 points.
    #x = np.linspace(0.0, 1.0, N)
    #y = 1*np.sin(10.0 * 2.0*np.pi*x) + 9*np.sin(20.0 * 2.0*np.pi*x)


    # Original Bandpass
    fs = 512.0
    fso2 = fs/2
    #Nd,wn = buttord(wp=[9/fso2,11/fso2], ws=[8/fso2,12/fso2],
    #   gpass=3.0, gstop=40.0)
    #b,a = butter(Nd,wn,'band')
    #y = filtfilt(b,a,y)

    y = butter_bandpass_filter(y, 128.0, 135.0, fs, order=6)


    yf = fft(y)
    #xf = np.linspace(0.0, int(1.0/(2.0*T)), int(N/2))
    #import matplotlib.pyplot as plt
    #plt.plot(xf, 2.0/N * np.abs(yf[0:int(N/2)]))
    #plt.axis((0,60,0,1))
    #plt.grid()
    #plt.show()

    return np.sum(np.abs(yf[0:int(N/2)]))

In [11]:
# Medida: Crest Factor

def crest_factor(x):
    return np.max(np.abs(x))/np.sqrt(np.mean(np.square(x)))

In [12]:
# Medidas: activity, morbidity, complexity

def hjorth(a):
    r"""
    Compute Hjorth parameters [HJO70]_.
    .. math::
        Activity = m_0 = \sigma_{a}^2
    .. math::
        Complexity = m_2 = \sigma_{d}/ \sigma_{a}
    .. math::
        Morbidity = m_4 =  \frac{\sigma_{dd}/ \sigma_{d}}{m_2}
    Where:
    :math:`\sigma_{x}^2` is the mean power of a signal :math:`x`. That is, its variance, if it's mean is zero.
    :math:`a`, :math:`d` and :math:`dd` represent the original signal, its first and second derivatives, respectively.
    .. note::
        **Difference with PyEEG:**
        Results is different from [PYEEG]_ which appear to uses a non normalised (by the length of the signal) definition of the activity:
        .. math::
            \sigma_{a}^2 = \sum{\mathbf{x}[i]^2}
        As opposed to
        .. math::
            \sigma_{a}^2 = \frac{1}{n}\sum{\mathbf{x}[i]^2}
    :param a: a one dimensional floating-point array representing a time series.
    :type a: :class:`~numpy.ndarray` or :class:`~pyrem.time_series.Signal`
    :return: activity, complexity and morbidity
    :rtype: tuple(float, float, float)
    Example:
    >>> import pyrem as pr
    >>> import numpy as np
    >>> # generate white noise:
    >>> noise = np.random.normal(size=int(1e4))
    >>> activity, complexity, morbidity = pr.univariate.hjorth(noise)
    """

    first_deriv = np.diff(a)
    second_deriv = np.diff(a,2)

    var_zero = np.mean(a ** 2)
    var_d1 = np.mean(first_deriv ** 2)
    var_d2 = np.mean(second_deriv ** 2)

    activity = var_zero
    morbidity = np.sqrt(var_d1 / var_zero)
    complexity = np.sqrt(var_d2 / var_d1) / morbidity

    return activity, morbidity, complexity

In [13]:
# Medida: Petrosian Fractal Dimension

def pfd(a):
    r"""
    Compute Petrosian Fractal Dimension of a time series [PET95]_.
    It is defined by:
    .. math::
        \frac{log(N)}{log(N) + log(\frac{N}{N+0.4N_{\delta}})}
    .. note::
        **Difference with PyEEG:**
        Results is different from [PYEEG]_ which implemented an apparently erroneous formulae:
        .. math::
            \frac{log(N)}{log(N) + log(\frac{N}{N}+0.4N_{\delta})}
    Where:
    :math:`N` is the length of the time series, and
    :math:`N_{\delta}` is the number of sign changes.
    :param a: a one dimensional floating-point array representing a time series.
    :type a: :class:`~numpy.ndarray` or :class:`~pyrem.time_series.Signal`
    :return: the Petrosian Fractal Dimension; a scalar.
    :rtype: float
    Example:
    >>> import pyrem as pr
    >>> import numpy as np
    >>> # generate white noise:
    >>> noise = np.random.normal(size=int(1e4))
    >>> pr.univariate.pdf(noise)
    """

    diff = np.diff(a)
    # x[i] * x[i-1] for i in t0 -> tmax
    prod = diff[1:-1] * diff[0:-2]

    # Number of sign changes in derivative of the signal
    N_delta = np.sum(prod < 0)
    n = len(a)

    return np.log(n)/(np.log(n)+np.log(n/(n+0.4*N_delta)))

In [24]:
# Create an empty dataframe for dffinal
columns = ['target', 'ptp', 'rms', 'cf'
           , 'entropy', 'activity', 'complexity'
           , 'morbidity', 'fractal', 'psd9']

dffinal = pd.DataFrame(columns=columns)

# Define the window size
window_size = 512

# Function to compute measures for a given window
def compute_measures(data):
    # Add your logic here to compute measure1 to measure9
    # For example, assuming 'data' is a pandas Series for the 'eeg' column:
    ptp = abs(np.max(data)) + abs(np.min(data))
    rms = np.sqrt(np.mean(data**2))
    cf = crest_factor(data)
    entropy = stats.entropy(list(Counter(data).values()), base=2)
    activity, complexity, morbidity = hjorth(data)
    fractal = pfd(data)
    psd(data)
    # Add more measures as needed
    return ptp,rms,cf,entropy,activity, complexity, morbidity,fractal,psd # Add other measures as needed

# Populate dffinal using data from df1
for i in range(len(df1) - window_size + 1):
    window_data = df1['eeg'].iloc[i : i + window_size]
    measures = compute_measures(window_data)
    dffinal = dffinal.append({'target': 0
                              , 'ptp': measures[0]
                              , 'rms': measures[1]
                              , 'cf': measures[2]
                              , 'entropy': measures[3]
                              , 'activity': measures[4]
                              , 'complexity': measures[5]
                              , 'morbidity': measures[6]
                              , 'fractal': measures[7]
                              , 'psd': measures[8]
                              }
                             , ignore_index=True)

# Populate dffinal using data from df2
for i in range(len(df2) - window_size + 1):
    window_data = df2['eeg'].iloc[i : i + window_size]
    measures = compute_measures(window_data)
    dffinal = dffinal.append({'target': 1
                              , 'ptp': measures[0]
                              , 'rms': measures[1]
                              , 'cf': measures[2]
                              , 'entropy': measures[3]
                              , 'activity': measures[4]
                              , 'complexity': measures[5]
                              , 'morbidity': measures[6]
                              , 'fractal': measures[7]
                              , 'psd': measures[8]
                              }
                             , ignore_index=True)

# Reset index of the final dataframe
dffinal.reset_index(drop=True, inplace=True)

In [25]:
dffinal.shape

(60668, 11)

In [26]:
dffinal.head()

,target,ptp,rms,cf,entropy,activity,complexity,morbidity,fractal,psd9,psd
0,0,1055,145.506732,4.219736,7.352986,21172.208984,0.188514,5.499801,1.020167,NaN,<function psd at 0x000001F30118E1E0>
1,0,1072,146.862007,4.180795,7.359324,21568.449219,0.186804,5.548790,1.020052,NaN,<function psd at 0x000001F30118E1E0>
2,0,1106,148.430658,4.136612,7.367413,22031.660156,0.185107,5.592249,1.019937,NaN,<function psd at 0x000001F30118E1E0>
3,0,1138,150.193528,4.088059,7.367413,22558.095703,0.183148,5.644323,1.019937,NaN,<function psd at 0x000001F30118E1E0>
4,0,1159,152.089419,4.037099,7.372794,23131.191406,0.180834,5.718295,1.019937,NaN,<function psd at 0x000001F30118E1E0>


In [27]:
dffinal.describe()

,rms,cf,entropy,activity,complexity,morbidity,fractal,psd9
count,60668.000000,60668.000000,60668.000000,60668.000000,60668.000000,60668.000000,60668.000000,0.0
mean,202.681089,3.301256,7.804700,44471.120171,0.122871,8.655943,1.018172,NaN
std,58.237037,0.556903,0.411772,20337.960422,0.066470,2.007113,0.001531,NaN
min,34.916949,1.982388,6.366749,1219.193359,0.058728,1.960549,1.012918,NaN
25%,188.134446,2.943870,7.635770,35394.569824,0.087742,7.960362,1.017153,NaN
50%,212.062518,3.273976,7.912101,44970.511719,0.106085,8.991340,1.018317,NaN
75%,237.413896,3.596657,8.094257,56365.357910,0.125235,9.968221,1.019360,NaN
max,345.913482,8.218705,8.448871,119656.136719,0.555771,13.552365,1.021890,NaN


In [28]:
df1.shape[0]+df2.shape[0]

61690

In [30]:
df1.shape[0]

30850

In [29]:
dffinal.target.value_counts()

0    30339
1    30329
Name: target, dtype: int64

In [31]:
dffinal.to_csv('datos/dffinal.csv', index=False)